# Retrieve Route Attributes from Dijkstra Results
We have a list of edges or turns (depending on the settings) from Dijkstra, and we need to get the route attributes (how many turns, how much feet up, miles of bike facility, etc) to get a means of understanding this trip.

For link attributes we the linkid and direction of travel (because of elevation).
For turn attributes we just need linkid to linkid.

In [28]:
from pathlib import Path
import time
import geopandas as gpd
import numpy as np
import pickle
import networkx as nx
from tqdm import tqdm
from shapely.ops import MultiLineString
import pandas as pd
import sys

# #TODO will need to fix this
# sys.path.insert(0,str(Path.cwd().parent))
# from network.src import modeling_turns

In [29]:
fp = Path.home() / "Documents/BikewaySimData/Projects/gdot"

In [30]:
with Path(fp/'gps_traces/cleaned_traces.pkl').open('rb') as fh:
    trips_df = pickle.load(fh)[1]
trips_df.columns

Index(['tripid', 'initial_start_time', 'initial_end_time', 'initial_duration',
       'initial_total_points', 'initial_avg_accuracy', 'total_points',
       'duration', 'max_delta_time', 'mean_delta_time', 'max_distance_ft',
       'avg_distance_ft', 'total_distance_ft', 'first_to_last_ft',
       'max_speed_mph', 'min_speed_mph', 'avg_speed_mph'],
      dtype='object')

# Case 1: Already have link sequence (matched)

In [31]:
with Path('D:/matched_traces/match_testing.pkl').open('rb') as fh:
    matched_traces_dict = pickle.load(fh)

In [32]:
# #TODO deal with having multiple matching networks (two, oneway, etc?)
# #Matching network
# # with (fp / 'chosen.pkl').open('rb') as fh:
# #     (df_edges,pseudo_df,pseudo_G) = pickle.load(fh)

# edges = gpd.read_file(fp/'networks/final_network.gpkg',layer="final_network")
# edges.reset_index(inplace=True)
# edges.rename(columns={'index':'linkid'},inplace=True)
# #edges = edges[['A','B','linkid','oneway','tunnel','bridge','geometry']]

# # #turn oneway into boolean (figure out why this is changing)
# edges['oneway'] = edges['oneway'] == 1

# #turn bridge and tunnel to boolean
# edges['tunnel'] = edges['tunnel'].notna()
# edges['bridge'] = edges['bridge'].notna()
# #TODO add ability to model wrongway
# df_edges, pseudo_df, pseudo_G = modeling_turns.create_pseudo_dual_graph(edges,'A','B','linkid','oneway',True)

In [33]:
with (fp / 'chosen.pkl').open('rb') as fh:
    df_edges,pseudo_df,pseudo_G = pickle.load(fh)

In [34]:
df_edges.columns

Index(['source', 'target', 'reverse_link', 'azimuth', 'linkid', 'osmid',
       'link_type', 'name', 'oneway', 'length_ft', 'bridge', 'tunnel',
       'link_type', 'highway', 'speedlimit_range_mph', 'lanes_per_direction',
       'bike_facility_type', 'ascent_m', 'ascent_grade', '(0,2]_ascent',
       '(2,4]_ascent', '(4,6]_ascent', '(6,10]_ascent', '(10,15]_ascent',
       '(15,inf]_ascent'],
      dtype='object')

In [35]:
#TODO this shows up twice FIX IT
df_edges = df_edges.loc[:,~df_edges.columns.duplicated()].copy()


In [36]:
matched_traces_dict.keys()

dict_keys([6148, 4100, 3592, 33804, 8204, 5133, 18451, 7703, 17433, 4121, 32282, 25120, 548, 16421, 2599, 10793, 555, 13355, 13358, 17974, 27705, 3647, 17473, 8258, 3139, 9797, 30277, 28749, 9806, 30799, 6737, 82, 11347, 11860, 11858, 12374, 3159, 3671, 15958, 7257, 29281, 24168, 15466, 32874, 27245, 31854, 17007, 16498, 2675, 33908, 3189, 4725, 9336, 121, 8825, 17534, 9860, 30345, 33929, 3209, 4745, 1162, 29837, 8335, 6800, 30352, 27283, 4243, 8348, 32417, 24737, 11432, 6312, 1194, 7341, 6830, 9393, 4277, 33976, 1212, 8893, 15552, 3265, 30913, 8900, 14022, 27850, 8402, 13526, 5338, 32476, 27871, 14050, 5859, 28388, 10982, 2279, 12008, 2281, 742, 235, 8429, 2804, 10998, 3322, 24828, 14077, 2814, 3333, 262, 12039, 1799, 9989, 12554, 14604, 29452, 7951, 1811, 4888, 7964, 15139, 16676, 6953, 17708, 34095, 30000, 305, 7473, 26422, 17207, 2871, 26426, 4412, 12098, 11590, 32074, 32076, 7503, 3413, 12119, 3419, 12636, 8029, 15710, 11618, 10596, 3941, 10601, 15727, 26479, 29557, 1400, 26488, 5

Test version

In [37]:
# tripid = 29837

# edges = matched_traces_dict[tripid]['edges']

# # make list of edges and turns
# list_of_edges = list(zip(edges['linkid'],edges['reverse_link']))
# df_edges['tup'] = list(zip(df_edges['linkid'],df_edges['reverse_link']))
# chosen_links = df_edges.set_index('tup').loc[list_of_edges]

# chosen_links['reverse_link']
# list_of_turns = [(list_of_edges[i][0],list_of_edges[i][1],list_of_edges[i+1][0],list_of_edges[i+1][1]) for i in range(0,len(list_of_edges)-1)]
# chosen_turns = pseudo_df.set_index(['source_linkid','source_reverse_link','target_linkid','target_reverse_link']).loc[list_of_turns]

# # # cols_to_summarize = ['highway','oneway','highway',
# # #        'oneway', 'bridge', 'tunnel', 'bl', 'pbl', 'mu',
# # #        'speedlimit_range_mph',
# # #        'lanes_per_direction', 'FROM_LANES', 'TO_LANES', 'join_bearing',
# # #        'percent_overlap', 'bearing_diff', 'match name', 'name_check',
# # #        'overlap_check', 'bearing_check', 'final_check', 'signal_A', 'signal_B',
# # #        'osm_url', 'rise_m', 'maxrise_m', 'up_grade', 'max_grade', 'down_m',
# # #        'minrise_m', 'down_grade', 'min_grade', 'geometry']
# # bool_cols = ['bl','mu','pbl']
# # count_cols = ['bridge','tunnel']
# # sum_cols = ['length','rise_m','down_m']
# # length_cols = ['link_type','highway','speedlimit_range_mph','lanes_per_direction']
# # elevation_cats = ['up_grade','down_grade']
# # keep_cols = bool_cols + count_cols + length_cols
# chosen_links.columns
# #intialize summary dict
# summary_attributes = {}

# # Get Link Attributes

# # #boolean columns to summarize
# # bool_cols = ['bl','mu','pbl']
# # for bool_col in bool_cols:
# #     total_length = chosen_links.loc[chosen_links[bool_col] == 1,'length_ft'].sum()
# #     summary_attributes[bool_col] = np.round(total_length/chosen_links['length_ft'].sum(),2)

# #instance columns to summarize
# count_cols = ['bridge','tunnel']
# for count_col in count_cols:
#     summary_attributes[count_col] = (chosen_links[count_col]==True).sum()

# #pct of route columns to summarize
# cols = ['link_type','highway','bike_facility_type','speedlimit_range_mph','lanes_per_direction']
# for col in cols:
#     for unique_val in chosen_links[col].unique():
#         if (unique_val != None) | (unique_val == np.nan):
#             total_length = chosen_links[chosen_links[col] == unique_val]['length_ft'].sum()
#         else:
#             continue
#         if isinstance(unique_val,str) == False:
#             unique_val = str(unique_val)
#         summary_attributes[col+'.'+unique_val] = np.round(total_length/chosen_links['length_ft'].sum(),2)

# # signalized

# #turn attributes (these are all instances)
# summary_attributes['signalized'] = chosen_turns['signalized'].sum()
# summary_attributes['unsignalized'] = chosen_turns['unsignalized'].sum()

# #turn type
# turn_dict = chosen_turns['turn_type'].value_counts().to_dict()
# summary_attributes.update(turn_dict)

# summary_attributes


In [38]:

summary_dict = {}

for tripid in matched_traces_dict.keys():

    if isinstance(matched_traces_dict[tripid],str):
        continue

    edges = matched_traces_dict[tripid]['edges']

    # make list of edges and turns
    list_of_edges = list(zip(edges['linkid'],edges['reverse_link']))
    df_edges['tup'] = list(zip(df_edges['linkid'],df_edges['reverse_link']))
    chosen_links = df_edges.set_index('tup').loc[list_of_edges]
    list_of_turns = [(list_of_edges[i][0],list_of_edges[i][1],list_of_edges[i+1][0],list_of_edges[i+1][1]) for i in range(0,len(list_of_edges)-1)]
    chosen_turns = pseudo_df.set_index(['source_linkid','source_reverse_link','target_linkid','target_reverse_link']).loc[list_of_turns]
    chosen_links.columns
    #intialize summary dict
    summary_attributes = {}

    #trip distance
    summary_attributes['trip_distance_ft'] = chosen_links['length_ft'].sum()

    #instance columns to summarize
    count_cols = ['bridge','tunnel']
    for count_col in count_cols:
        summary_attributes[count_col] = (chosen_links[count_col]==True).sum()

    #general elevation
    total_ascent = (chosen_links['ascent_m'].sum() / 3.28).round(0)
    summary_attributes['ascent_ft'] = total_ascent 
    summary_attributes['avg_ascent_grade'] = (total_ascent / chosen_links['length_ft'].sum() * 100).round(1)

    #elevation broken by segment
    elev_cols = ['(0,2]_ascent','(2,4]_ascent', '(4,6]_ascent', '(6,10]_ascent', '(10,15]_ascent','(15,inf]_ascent']
    for elev_col in elev_cols:
        total_length = (chosen_links[elev_col].sum() / 3.28).round(0)
        summary_attributes[elev_col+'_ft'] = total_length

    #pct of route columns to summarize
    cols = ['link_type','highway','bike_facility_type','speedlimit_range_mph','lanes_per_direction']
    for col in cols:
        #make a summary column for every unique value in that column
        for unique_val in chosen_links[col].unique():
            if (unique_val != None) | (unique_val == np.nan):
                total_length = chosen_links[chosen_links[col] == unique_val]['length_ft'].sum()
            else:
                continue
            if isinstance(unique_val,str) == False:
                unique_val = str(unique_val)
            summary_attributes[col+'.'+unique_val] = np.round(total_length/chosen_links['length_ft'].sum(),2)

    # signalized and turns
    summary_attributes['signalized'] = (chosen_turns['signalized']==True).sum()
    summary_attributes['unsignalized'] = (chosen_turns['unsignalized']==True).sum()
    turn_dict = chosen_turns['turn_type'].value_counts().to_dict()
    summary_attributes.update(turn_dict)

    summary_dict[tripid] = summary_attributes



In [39]:
#turn into geodataframe
trips_df_info = pd.DataFrame.from_dict(summary_dict,orient='index')
trips_df_info.fillna(0,inplace=True)

In [40]:
trips_df = trips_df.merge(trips_df_info,left_on='tripid',right_index=True)

In [41]:
#TODO find the visualiztion code that we had already made

In [42]:
# list_of_edges = list(zip(edges['linkid'],edges['reverse_link']))
# list_of_turns = [(list_of_edges[i][0],list_of_edges[i][1],list_of_edges[i+1][0],list_of_edges[i+1][1]) for i in range(0,len(list_of_edges)-1)]

# chosen_links = df_edges.set_index(['linkid','reverse_link'],drop=False).loc[list_of_edges]

# chosen_links['bridge'].value_counts()

In [43]:
#add user info
trip_and_user = pd.read_pickle(fp/'gps_traces/trip_and_user.pkl')

trips_df = trips_df_info.merge(trip_and_user,left_index=True,right_on='tripid')

In [45]:
trips_df.to_csv(fp/'all_attrs.csv',index=False)

In [17]:
#instead of visualizing here visualize elsewhere?

In [18]:

#import stochastic_optimization


In [19]:
# fp = Path.home() / 'Documents/BikewaySimData/Projects/gdot'
# #fp = Path.home() / 'Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/BikewaySim/Data'

# with (fp / 'impedance_calibration.pkl').open('rb') as fh:
#     (df_edges,pseudo_df,pseudo_G) = pickle.load(fh)

In [20]:
# Retrieve link/turn costs
# default below is link distance
# link_costs = dict(zip(list(zip(df_edges['source'],df_edges['target'],df_edges['linkid'])),df_edges['length_ft']))
# tup = list(zip(pseudo_df['source_A'],pseudo_df['source_B'],pseudo_df['source_linkid']))
# pseudo_df['source_cost'] = list(map(link_costs.get,tup))
# tup = list(zip(pseudo_df['target_A'],pseudo_df['target_B'],pseudo_df['target_linkid']))

# pseudo_df['target_cost'] = list(map(link_costs.get,tup))
# pseudo_df['total_cost'] = pseudo_df['source_cost'] + pseudo_df['target_cost'] #+turn_cost

# costs = pseudo_df.groupby(['source','target'])['total_cost'].min()
# nx.set_edge_attributes(pseudo_G,values=costs,name='weight')
# source = list(pseudo_G.nodes())[0]
# target = list(pseudo_G.nodes())[420]
# print(source,target)
# import networkx as nx
# length, edge_list = nx.single_source_dijkstra(pseudo_G,source,target,weight="weight")
# turn_list = [[edge_list[i][0],edge_list[i][1],edge_list[i+1][0],edge_list[i+1][1]] for i in range(len(edge_list)-1)]

# turn_cols = ['turn_type','signalized_left_straight','unsignalized_left_straight_nonlocal']
# linkid_cols = ['source_linkid','source_reverse_link','target_linkid','target_reverse_link']
# chosen_turns = pseudo_df.set_index(['source_A','source_B','target_A','target_B'],drop=False).loc[turn_list,linkid_cols+turn_cols]

# tripid = 302

# #make a single row dataframe to attach to trips_df
# stats_dict = {}
# stats_dict[tripid] = {
#     'tripid':tripid,
#     'signalized_left_straight': chosen_turns['signalized_left_straight'].sum(),
#     'unsignalized_left_straight_nonlocal': chosen_turns['unsignalized_left_straight_nonlocal'].sum()
# }
# turn_dict = chosen_turns['turn_type'].value_counts().to_dict()
# stats_dict[tripid].update(turn_dict)

# Case 1: Need to run shortest paths to create link sequence

In [ ]:
# source_links = chosen_turns[['source_linkid','source_reverse_link']]
# target_links = chosen_turns[['target_linkid','target_reverse_link']]
# source_links.columns = ['linkid','reverse_link']
# target_links.columns = ['linkid','reverse_link']
# linkids = pd.concat([source_links,target_links],ignore_index=True).drop_duplicates()
# chosen_links = df_edges.merge(linkids,on=['linkid','reverse_link'])

In [ ]:
#TODO recalculate bearing
#create pseudo graph for modeling turns
df_edges, pseudo_links, pseudo_G = modeling_turns.create_pseudo_dual_graph(links,'A','B','linkid','oneway')


In [ ]:
source = list(pseudo_G.nodes())[0]
target = list(pseudo_G.nodes())[420]

In [ ]:
source = (68209677, 68209675)
target = (69200243, 69465418)

import networkx as nx
length, path = nx.single_source_dijkstra(pseudo_G,source,target)

In [ ]:
edge_list = [(linkids[i],linkids[i+1]) for i in range(len(linkids)-1)]
edge_list

In [ ]:
edge_list = [(*path[i],*path[i+1]) for i in range(len(path)-1)]
edge_list

In [ ]:
pseudo_links.columns

In [ ]:
pseudo_links.set_index(['source_A','source_B','target_A','target_B']).loc[edge_list,'turn_type'].value_counts()